In [ ]:
import collections
import pickle
import pandas as pd
import os
import peppi_py
import tree
import json
import io
import math
import tqdm.notebook
import functools
import sqlite3

from slippi_db import utils, preprocessing, parse_peppi

In [ ]:
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

In [ ]:
root = '/linusr/vlad/SSBM/Replays/'
parsed_path = os.path.join(root, 'parsed.sqlite')

In [ ]:
%%time

conn = sqlite3.connect(parsed_path)
df = pd.read_sql_query('SELECT * FROM replays', conn, dtype_backend='numpy_nullable')
conn.close()

In [ ]:
@functools.cache
def compact_raw(raw: str) -> str:
    if raw.startswith('Players/'):
        return raw.split('/')[1]
    if raw.startswith('Phillip/'):
        return 'Phillip'
    return raw

df['compact_raw'] = df['raw'].map(compact_raw)

In [ ]:
df['valid'] = df['valid'].astype(bool)  # sqlite doesn't have booleans

In [ ]:
df.groupby('compact_raw')['valid'].mean()

In [ ]:
valid_df = df[df['valid']]

In [ ]:
prefixes = [f'p{i}_' for i in range(2)]

def get_player_from_key(key: str):
    for i, prefix in enumerate(prefixes):
        if key.startswith(prefix):
            return i
    return None

common_keys = []
player_keys = [[] for _ in prefixes]

for key in df.keys():
    player = get_player_from_key(key)
    if player is None:
        common_keys.append(key)
    else:
        player_keys[player].append(key)

def get_player_df(df, player: int):
    to_drop = player_keys[1 - player]
    
    key_mapping = {}
    for key in player_keys[player]:
        key_mapping[key] = key.removeprefix(prefixes[player])
    
    return df.drop(columns=to_drop).rename(columns=key_mapping)

In [ ]:
per_player_df = pd.concat([get_player_df(valid_df, i) for i in range(2)])

In [ ]:
per_player_df.keys()

In [ ]:
for key in ['name_tag', 'netplay_name', 'netplay_code']:
    per_player_df[key] = per_player_df[key].fillna('')

In [ ]:
no_char = per_player_df[per_player_df['character'].isna()]
no_char['compact_raw'].value_counts()

In [ ]:
per_player_df.fillna({'character': melee.Character.FOX.value}, inplace=True)

In [ ]:
import typing as tp
import itertools
import melee
from slippi_ai import nametags

In [ ]:
def map_df(f, keys):
    return list(map(f, *(df[k] for k in keys)))

def get_name(row: dict):
    player_meta = dict(
        character=row['character'],
        name_tag=row['name_tag'],
        netplay=dict(
            name=row['netplay_name'],
            code=row['netplay_code'],
        ),
    )
    # TODO: add raw
    name = nametags.name_from_metadata(player_meta, raw=row['raw'])
    return nametags.normalize_name(name)

per_player_df['name'] = [
    get_name(row)
    for _, row in
    tqdm.notebook.tqdm(per_player_df.iterrows(), total=len(per_player_df))
]

In [ ]:
per_player_df['has_name'] = per_player_df['name'] != ''

In [ ]:
per_player_df.groupby('compact_raw')['has_name'].mean().sort_values()

In [ ]:
cody_df = per_player_df[per_player_df['compact_raw'] == 'Cody']
len(cody_df)

In [ ]:
cody_df['name'].value_counts()

In [ ]:
noname_cody_df = cody_df[(cody_df['name'] == '') & (cody_df['char'] == 'FOX')]
len(noname_cody_df)

In [ ]:
noname_cody_df['slippi_version'].value_counts()

In [ ]:
noname_cody_df['playedOn'].value_counts()

In [ ]:
by_version = noname_cody_df.groupby('slippi_version').head(10).reset_index()

In [ ]:
import peppi_py
import tempfile

def read_game(f: utils.LocalFile):
    with f.extract('/dev/shm') as path:
        return peppi_py.read_slippi(path, skip_frames=True)

def metadata_has_name(metadata):
    for port in ['0', '1']:
        if port not in metadata['players']:
            return False
        if metadata['players'][port]['names'].get('code') is None:
            return False
    return True

In [ ]:
names = []
for _, row in tqdm.notebook.tqdm(by_version.iterrows(), total=len(by_version)):
    zf = utils.ZipFile(os.path.join(root, '../Raw', row['raw']), row['filename'] + 'z')
    assert os.path.isfile(zf.root)
    # metadata.append(flatten(read_game(zf).metadata))
    game = read_game(zf)
    has_name = metadata_has_name(game.metadata)
    if has_name:
        name = game.metadata['players']['0']['names']['code']
    else:
        name = None
    names.append(name)

In [ ]:
by_version['meta_name'] = names
by_version['has_name'] = ~by_version['meta_name'].isnull()

In [ ]:
by_version.groupby('slippi_version')['has_name'].mean()

In [ ]:
v3_18_0 = by_version[by_version['slippi_version'] == (3, 18, 0)]
v3_18_0

In [ ]:
by_version.groupby('slippi_version')['netplay'].apply(lambda x: x.isnull().mean())

In [ ]:
game.metadata

In [ ]:
game.start.players[0].netplay